# Data Encoding

This notebook provides a short walkthrough of some of the data encoding features of the `sharrow` library.

In [ ]:
# HIDDEN
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
from io import StringIO

import sharrow as sh
sh.__version__

In [ ]:
# check versions
import packaging
assert packaging.version.parse(xr.__version__) >= packaging.version.parse("0.20.2")

## Example Data

We'll begin by importing some example data to work with.  We'll be using 
some test data taken from the MTC example in the ActivitySim project. For
this data encoding walkthrough, we'll focus on the
skims containing transportation level of service information for travel around
a tiny slice of San Francisco.

We'll load them as a multi-dimensional `xarray.Dataset` — or, more exactly, a 
`sharrow.Dataset`, which is a subclass from the xarray version that adds some 
useful features, including compatability with automatic tools for recoding data.

In [ ]:
skims = sh.example_data.get_skims()
skims

Because sharrow uses the `xarray.Dataset` format to work with data, individual 
variables in each Dataset can be encoded in different data types.
For example, automobile travel times can be stored with 
high(er) precision floating point numbers, while transit 
fares, which vary less and have a narrower range, can be 
stored with lower precision.  This allows a user to choose 
the most efficient encoding for each variable, if desired. 

## Fixed Point Encoding

Very often, data (especially skim matrixes like here) can be expressed adequately 
with far less precicion than a standard 32-bit floating point representation allows.
In these cases, it may be beneficial to store this 
data with "fixed point" encoding, which is also 
sometimes called scaled integers.

Instead of storing values as 32-bit floating point values, 
they could be multiplied by a scale factor (e.g., 100) 
and then converted to 16-bit integers. This uses half the
RAM and can still express any value (to two decimal point 
precision) up to positive or negative 327.68.  If the lowest 
values in that range are never needed, it can also be shifted,
moving both the bottom and top limits by a fixed amount. Then, 
for a particular scale $\mu$ and shift $\xi$ (stored in metadata),
from any array element $i$ the implied (original) value $x$ 
can quickly be recovered by evaluating $(i / \mu) - \xi$.

Sharrow includes a pair of functions to encode and decode arrays in
this manner. These functions also attach the necessary metadata
to the Dataset objects, so that later when we construct `sharrow.Flow` 
instances, they can decode arrays automatically.

In [ ]:
from sharrow.digital_encoding import array_encode, array_decode

The distance data in the skims is a great candidate for fixed point
of encoding.  We can peek at the top corner of this array:

In [ ]:
skims.DIST.values[:2,:3]

The data are all small(ish) values with two decimal point fixed
precision, so we can probably efficiently encode this data by scaling by 100.
If we're not sure, we can confirm by checking the range of values, to make
sure it fits inside the 16-bit integers we're hoping to use.

In [ ]:
skims.DIST.values.min(), skims.DIST.values.max()

That's a really small range because this is only test data.  But even 
the full-scale MTC skims spanning the entire region don't contain distances
over 300 miles.

We can create a new DataArray and apply fixed point encoding using the
`array_encode` function.

In [ ]:
distance_encoded = array_encode(skims.DIST, scale=0.01, offset=0)
distance_encoded.values[:2,:3]

In [ ]:
# TEST encoding
assert distance_encoded.dtype == np.int16
np.testing.assert_array_equal(
    distance_encoded.values[:2,:3],
    np.array([[12, 24, 44], [37, 14, 28]], dtype=np.int16)
)
assert distance_encoded.attrs['digital_encoding'] == {'scale': 0.01, 'offset': 0, 'missing_value': None}

We can apply that function for any number of variables in the skims, and
create a new Dataset that includes the encoded arrays.

In [ ]:
skims_encoded = skims.assign(
    {'DIST': array_encode(skims.DIST, scale=0.01, offset=0)}
)

To manage the digital encodings across an entire dataset, sharrow implements
a `digital_encoding` accessor.  You can use it to apply encodings to one or more
variables in a simple fashion.

In [ ]:
skims_encoded = skims_encoded.digital_encoding.set(['DISTWALK', 'DISTBIKE'], scale=0.01, offset=0)

And you can review the encodings for every variable in the dataset like this:

In [ ]:
skims_encoded.digital_encoding.info()

In [ ]:
# TEST
assert skims_encoded.digital_encoding.info() == {
 'DIST': {'scale': 0.01, 'offset': 0, 'missing_value': None},
 'DISTBIKE': {'scale': 0.01, 'offset': 0, 'missing_value': None},
 'DISTWALK': {'scale': 0.01, 'offset': 0, 'missing_value': None},
}

To demonstrate that the encoding works transparently with a `Flow`,
we can construct a simple flow that extracts the distance and 
square of distance for the top corner of values we looked at above.

First we'll do so for a flow with the original float32 encoded skims.

In [ ]:
pairs = pd.DataFrame({'orig': [0,0,0,1,1,1], 'dest': [0,1,2,0,1,2]})
tree = sh.DataTree(
    base=pairs, 
    skims=skims.drop_dims('time_period'), 
    relationships=(
        "base.orig -> skims.otaz",
        "base.dest -> skims.dtaz",
    ),
)
flow = tree.setup_flow({'d1': 'DIST', 'd2': 'DIST**2'})
arr = flow.load()
arr

We can do the same for the encoded skims, and we get exactly the
same result, even though the encoded skims use less RAM.

In [ ]:
tree_enc = sh.DataTree(
    base=pairs, 
    skims=skims_encoded.drop_dims('time_period'), 
    relationships=(
        "base.orig -> skims.otaz",
        "base.dest -> skims.dtaz",
    ),
)
flow_enc = tree_enc.setup_flow({'d1': 'DIST', 'd2': 'DIST**2'})
arr_enc = flow_enc.load()
arr_enc

In [ ]:
# TEST
np.testing.assert_array_almost_equal(arr, arr_enc)

## Dictionary Encoding

For skim matrixes where the universe of all possible 
cell values can be adequately represented by just 255 
unique values, we can use an explicit mapping process
called "dictionary encoding", which works by storing 
those unique values in a tiny base array.  Then, in the 
main body of the skim data we only store offsets that point to 
positions in that base array. This reduces the marginal 
memory footprint of each array cell to just an 8 bit 
integer, reducing memory requirements by up to 75% for 
these arrays compared to float32's. This approach is 
particularly appropriate for many transit skims, as fares, 
wait times, and transfers can almost always be reduced 
to a dictionary encoding with no meaningful information 
loss.

For example, the `'WLK_LOC_WLK_FAR'` array containing fares
only has four unique values:

In [ ]:
np.unique(skims.WLK_LOC_WLK_FAR)

We can see various fares applied at different time periods if we
look at the top corner of the array:

In [ ]:
skims.WLK_LOC_WLK_FAR.values[:2,:3,:]

Once encoded, the array itself only contains offset pointers (small integers),
plus the original values stored in metadata.

In [ ]:
wlwfare_enc = array_encode(skims.WLK_LOC_WLK_FAR, bitwidth=8, by_dict=True)
wlwfare_enc.values[:2,:3,:]

In [ ]:
wlwfare_enc.attrs['digital_encoding']['dictionary']

In [ ]:
# TEST encoding
assert wlwfare_enc.dtype == np.uint8
np.testing.assert_array_equal(
    wlwfare_enc.values[:2,:3,:],
    np.array([[[0, 0, 0, 0, 0],
        [1, 2, 2, 1, 2],
        [1, 2, 2, 1, 2]],

       [[1, 1, 2, 2, 1],
        [0, 0, 0, 0, 0],
        [1, 2, 2, 1, 2]]], dtype=np.uint8)
)
np.testing.assert_array_equal(
    wlwfare_enc.attrs['digital_encoding']['dictionary'],
    np.array([   0.,  152.,  474.,  626.], dtype=np.float32)
)

If we want to recover the original data for analysis (other than in
a Flow, which can decode it automatically), we can use the `array_decode` function.

In [ ]:
array_decode(wlwfare_enc)

In [ ]:
# TEST
xr.testing.assert_equal(array_decode(wlwfare_enc), skims.WLK_LOC_WLK_FAR)

## Joint Dict Encoding

Dictionary encoding can be expanded to map multiple different variables
using the same underlying offsets array.  For large datasets with several
dimension lengths in the thousands, the offset array may constitute the 
vast majority of the memory usage, so sharing the same offsets for several
variables can result in huge reductions in the memory footprint.

The joint dictionary can be applied using the `set` method of the
`digital_encoding` accessor, giving a list of the variables to jointly encode:

In [ ]:
skims1 = skims.digital_encoding.set(
    ['WLK_LOC_WLK_FAR', 
     'WLK_EXP_WLK_FAR', 
     'WLK_HVY_WLK_FAR', 
     'DRV_LOC_WLK_FAR',
     'DRV_HVY_WLK_FAR',
     'DRV_EXP_WLK_FAR'],
    joint_dict=True,
)

A unique name is automatically generated for the join when `joint_dict` is set
to `True`.  Alternatively, the user can specify a name to use for the join by
giving a string input as the `joint_dict`.  Different sets of variables in the 
same Dataset can be grouped and encoded jointly, but each group must have a 
unique name.

In [ ]:
skims1 = skims1.digital_encoding.set(
    ['DISTBIKE', 
    'DISTWALK'],
    joint_dict="jointWB",
)

The resulting dataset adds a variable for each created group, which contains 
the offsets, and the named variables in the group are replaced with a new
one-dimension array with coordinating lengths.

In [ ]:
skims1

Skims encoded in this manner can be fed into sharrow and will compile and return 
the same results as if they were not encoded.  If you are mixing compiled flows
between encoded and unencoded Datasets (which should be unusual, but for the examples
in this notebook we've done it) you'll need to set the `hashing_level` to at least
2, to ensure you are matching the correct numba code with the encodings used in the
data.

In [ ]:
tree1 = sh.DataTree(
    base=pairs, 
    skims=skims1, 
    rskims=skims1, 
    relationships=(
        "base.orig -> skims.otaz",
        "base.dest -> skims.dtaz",
        "base.orig -> rskims.dtaz",
        "base.dest -> rskims.otaz",
    ),
)
flow1 = tree1.setup_flow({
    'd1': 'skims["WLK_LOC_WLK_FAR", "AM"]', 
    'd2': 'skims["WLK_LOC_WLK_FAR", "AM"]**2',
    'w1': 'skims.DISTWALK',
    'w2': 'skims.reverse("DISTWALK")',
    'w3': 'rskims.DISTWALK',
    'x1': 'skims.DIST',
    'x2': 'skims.reverse("DIST")',
}, hashing_level=2)
arr1 = flow1.load_dataframe()
arr1

In [ ]:
# TEST
assert (arr1 == np.array([[ 0.00000e+00,  0.00000e+00,  1.20000e-01,  1.20000e-01,  1.20000e-01,  1.20000e-01,  1.20000e-01],
       [ 4.74000e+02,  2.24676e+05,  2.40000e-01,  3.70000e-01,  3.70000e-01,  2.40000e-01,  3.70000e-01],
       [ 4.74000e+02,  2.24676e+05,  4.40000e-01,  5.70000e-01,  5.70000e-01,  4.40000e-01,  5.70000e-01],
       [ 1.52000e+02,  2.31040e+04,  3.70000e-01,  2.40000e-01,  2.40000e-01,  3.70000e-01,  2.40000e-01],
       [ 0.00000e+00,  0.00000e+00,  1.40000e-01,  1.40000e-01,  1.40000e-01,  1.40000e-01,  1.40000e-01],
       [ 4.74000e+02,  2.24676e+05,  2.80000e-01,  2.80000e-01,  2.80000e-01,  2.80000e-01,  2.80000e-01]],
      dtype=np.float32)).all().all()

In [ ]:
# TEST
assert skims1.digital_encoding.baggage(['WLK_LOC_WLK_FAR']) == {'joined_0_offsets'}
assert (skims1.iat(
    otaz=[0,1,2], dtaz=[0,0,0], time_period=[1,1,1],
    _name='WLK_LOC_WLK_FAR'
).to_series() == [0,152,474]).all()